In [1]:
import json 
import pandas as pd 



# Test model results 

In [6]:
test_model_results = []

with open("test_responses.jsonl",'r') as f : 
    for line in f: 
        if line.strip():
            data = json.loads(line)
            test_model_results.append(data)


In [7]:
df_test_model = pd.DataFrame(test_model_results)
df_test_model

,sample_id,test model,Encountered Problems,test model response
0,0,openai:gpt-3.5-turbo,False,Based on the information provided in the conte...
1,0,openai:gpt-4o,False,"Based on the context document, your risk facto..."


In [8]:
print(df_test_model['test model response'].iloc[0])

Based on the information provided in the context document, your risk factors for dementia include high blood pressure, lack of physical activity, poor diet, and low cognitive engagement. Cognitive engagement is important because it supports the development of a "cognitive reserve," which may protect against brain cell damage caused by dementia. Therefore, actively using your brain throughout your life may help reduce the risk of developing dementia.


In [9]:
print(df_test_model['test model response'].iloc[1])

Based on the context document, your risk factors for dementia include:

1. **Hearing Loss**: Having your ears blown out in the war may have led to hearing loss, which increases the risk of cognitive decline and dementia.
2. **Poor Diet**: Eating mostly garbage, which likely means an unhealthy diet high in saturated fat, sugar, and salt, can increase the risk of developing dementia.
3. **Depression**: Experiencing "a case of the sads pretty regular" suggests depression, which is associated with a higher risk of developing dementia.
4. **Social Isolation**: Although not explicitly mentioned, if your lifestyle leads to social isolation, it can increase the risk of dementia.

Regarding cognitive engagement, the document states that it is thought to support the development of a "cognitive reserve." This means that people who actively use their brains throughout their lives may be more protected against brain cell damage caused by dementia. Therefore, engaging in activities that stimulate yo

# Judges

In [10]:
judges_result = []

with open("judge_responses.jsonl",'r') as f : 
    for line in f:
        if line.strip():
            content = json.loads(line)
            judges_result.append(content)

In [11]:
df_judges = pd.DataFrame(judges_result)
df_judges

,judged_model,sample_id,judge model,Encountered Problems,judge model response,verdict
0,openai:gpt-3.5-turbo,0,anthropic:claude-3-5-sonnet-20240620,False,Sentence 1: Based on the information provided ...,Accurate
1,openai:gpt-3.5-turbo,0,google_vertexai:gemini-1.5-pro,False,"```json\n{""sentence"": ""Based on the informatio...",Accurate
2,openai:gpt-4o,0,anthropic:claude-3-5-sonnet-20240620,False,"Sentence 1: Based on the context document, you...",Accurate
3,openai:gpt-4o,0,google_vertexai:gemini-1.5-pro,False,"```json\n{""sentence"": ""Based on the context do...",Accurate


In [12]:
print(len(df_test_model))
print(len(df_judges))

2
4


# Explore some samples 

In [13]:
print(df_judges['judge model response'].iloc[0])

Sentence 1: Based on the information provided in the context document, your risk factors for dementia include high blood pressure, lack of physical activity, poor diet, and low cognitive engagement.
Sentence 1 label: Accurate

Sentence 2: Cognitive engagement is important because it supports the development of a "cognitive reserve," which may protect against brain cell damage caused by dementia.
Sentence 2 label: Accurate

Sentence 3: Therefore, actively using your brain throughout your life may help reduce the risk of developing dementia.
Sentence 3 label: Accurate

Final Answer: Accurate


In [14]:
print(df_judges['judge model response'].iloc[1])

```json
{"sentence": "Based on the information provided in the context document, your risk factors for dementia include high blood pressure, lack of physical activity, poor diet, and low cognitive engagement.", "label": "supported", "rationale": "The context lists each of these as risk factors for dementia.", "excerpt": "People who have consistent high blood pressure (hypertension) in mid-life (ages 45 to 65) are more likely to develop dementia compared to those with normal blood pressure.\n\nPhysical inactivity in later life (ages 65 and up) increases the risk of developing dementia.\n\nAn unhealthy diet, high in saturated fat, sugar, and salt, can increase the risk of developing many illnesses, including dementia and cardiovascular disease.\n\nCognitive engagement is thought to support the development of a\n“cognitive reserve”. This is the idea that people who actively use their brains throughout their lives may be more protected against brain cell damage caused by dementia."}
{"sent

# Overall Accuracy 

In [15]:
TEST_MODEL = 'openai:gpt-3.5-turbo' # feel free to checkout accuracy for other models 

In [18]:
df_judges[df_judges['judged_model']==TEST_MODEL].groupby('judge model')['verdict'].value_counts().unstack(fill_value=0)

verdict,Accurate
judge model,
anthropic:claude-3-5-sonnet-20240620,1
google_vertexai:gemini-1.5-pro,1


In [20]:
df_filtered = df_judges[df_judges['judged_model'] == TEST_MODEL]
counts = df_filtered.groupby('judge model')['verdict'].value_counts()
percentages = counts / counts.groupby(level=0).sum() * 100
result = percentages.unstack(fill_value=0)[["Accurate"]]

result

verdict,Accurate
judge model,
anthropic:claude-3-5-sonnet-20240620,100.0
google_vertexai:gemini-1.5-pro,100.0
